### Field report: PNL and hit rate analysis for 5 basic strategies.
### 17/03/17- v0.1
#### por mrg

In [93]:
#Load packages
Pkg.add("TimeSeries");Pkg.add("MarketData");Pkg.add("Plots");Pkg.add("PlotlyJS");
Pkg.add("Gadfly");Pkg.add("StatsBase.jl");Pkg.update();

INFO: Nothing to be done
INFO: Nothing to be done
INFO: Nothing to be done
INFO: Nothing to be done
INFO: METADATA might be out-of-date — you may not have the latest version of PlotlyJS
INFO: Use `Pkg.update()` to get the latest versions of your packages
INFO: Nothing to be done


LoadError: unknown package StatsBase.jl
 in macro expansion at .\pkg\entry.jl:53 [inlined]
 in (::Base.Pkg.Entry.##2#5{String,Base.Pkg.Types.VersionSet})() at .\task.jl:360

In [98]:
using TimeSeries, MarketData, Plots, PlotlyJS, StatsBase;

INFO: Recompiling stale cache file C:\Users\miguel\.julia\lib\v0.5\StatsBase.ji for module StatsBase.


In [12]:
typeof(ohlc)

TimeSeries.TimeArray{Float64,2,Date,Array{Float64,2}}

In [11]:
ohlc["Open"]

500x1 TimeSeries.TimeArray{Float64,1,Date,Array{Float64,1}} 2000-01-03 to 2001-12-31

             Open      
2000-01-03 | 104.88    
2000-01-04 | 108.25    
2000-01-05 | 103.75    
2000-01-06 | 106.12    
⋮
2001-12-26 | 21.35     
2001-12-27 | 21.58     
2001-12-28 | 21.97     
2001-12-31 | 22.51     


In [127]:
import TimeSeries.moving

In [128]:
# Función ventana
function moving{T}(ta::TimeArray{T,1}, f::Function, window::Int; padding::Bool=false)
    tstamps = padding ? ta.timestamp : ta.timestamp[window:end]
    vals    = zeros(ta.values[window:end])
    @inbounds for i=1:length(vals)
        vals[i] = f(ta.values[i:i+(window-1)])
    end
    padding && (vals = [NaN*ones(window-1); vals])
    TimeArray(tstamps, vals, ta.colnames, ta.meta)
end

function moving{T}(ta::TimeArray{T,2}, f::Function, window::Int; padding::Bool=false)
    tstamps = padding ? ta.timestamp : ta.timestamp[window:end]
    vals    = zeros(ta.values[window:end, :])
    @inbounds for i=1:size(vals,1), j=1:size(vals, 2)
        vals[i, j] = f(ta.values[i:i+(window-1), j])
    end
    padding && (vals = [NaN*ones(ta.values[1:(window-1), :]); vals])
    TimeArray(tstamps, vals, ta.colnames, ta.meta)
end

moving (generic function with 2 methods)

moving(TimeSeries.TimeArray{#T<:Any, 1, D<:Base.Dates.TimeType, A<:AbstractArray}, Function, Int64) in module TimeSeries at C:\Users\miguel\.julia\v0.5\TimeSeries\src\apply.jl:211 overwritten in module Main at In[128]:3.


In [96]:
#Nice, ya podemos tomar promedios en ventanas de tamaño N = 10
moving(ohlc,mean,10)

491x4 TimeSeries.TimeArray{Float64,2,Date,Array{Float64,2}} 2000-01-14 to 2001-12-31

             Open      High      Low       Close     
2000-01-14 | 100.692   103.981   96.001    98.782    
2000-01-18 | 100.304   103.331   95.876    97.982    
2000-01-19 | 100.041   103.144   96.095    98.388    
2000-01-20 | 101.216   104.238   97.145    99.338    
⋮
2001-12-26 | 21.062    21.507    20.618    21.065    
2001-12-27 | 21.033    21.54     20.651    21.123    
2001-12-28 | 21.081    21.685    20.797    21.266    
2001-12-31 | 21.259    21.868    20.971    21.417    


In [121]:
fs = (:mean)

for f in fs
@eval ($f) = moving(ohlc,$f,10)
end

LoadError: MethodError: no method matching start(::Symbol)[0m
Closest candidates are:
  start([1m[31m::SimpleVector[0m) at essentials.jl:170
  start([1m[31m::Base.MethodList[0m) at reflection.jl:258
  start([1m[31m::IntSet[0m) at intset.jl:184
  ...[0m

In [129]:
# Aquí se van a desarrollar las funciones estrategia